In [1]:
%load_ext autoreload
%autoreload 2     

In [2]:
from voting_mechanism_design.agents.quorum_badgeholder import QuorumBadgeholder, QuorumBadgeholderPopulation
from voting_mechanism_design.projects.project import Project, ProjectPopulation
from voting_mechanism_design.funds_distribution.threshold_and_aggregate import ThresholdAndAggregate
from voting_mechanism_design.sim import RoundSimulation

import matplotlib.pyplot as plt
import numpy as np
import itertools
from collections import defaultdict
from tqdm import tqdm
import pandas as pd
import warnings

# OP simulator configuration
from voting_mechanism_design.legacy.op_simulator import Simulation as OPSimulation

In [ ]:
#old framework
def run_single_simulation_old(n_badgeholders, badgeholder_expertise_vec, badgeholder_laziness_vec, n_projects, project_impact_vec, quorum, scoring_fn, min_vote_amt, max_vote_amt, max_funding, random_seed):
    np.random.seed(random_seed)
    
    simulation = Simulation()
    simulation.initialize_round(max_funding)
    simulation.randomize_voters(n_badgeholders, willingness_to_spend=min_vote_amt, laziness_factor=badgeholder_laziness_vec[0], expertise_factor=badgeholder_expertise_vec[0])
    simulation.randomize_projects(n_projects, coi_factor=0)
    simulation.simulate_voting()
    results = simulation.allocate_votes(scoring_fn, quorum=quorum, min_amount=min_vote_amt)
    
    return results

def run_n_simulations_old(
    n_sims,
    n_badgeholders=50,
    badgeholder_expertise_vec=None,
    badgeholder_laziness_vec=None,
    n_projects=100,
    project_impact_vec=None,
    quorum=5,
    scoring_fn='mean',
    min_vote_amt=1,
    max_vote_amt=16,
    max_funding=100,
    random_seed_start=1234,
    verbose=False,
):
    def run_simulation(ii):
        seed = random_seed_start + ii
        return run_single_simulation_old(
            n_badgeholders=n_badgeholders,
            badgeholder_expertise_vec=badgeholder_expertise_vec,
            badgeholder_laziness_vec=badgeholder_laziness_vec,
            n_projects=n_projects,
            project_impact_vec=project_impact_vec,
            quorum=quorum,
            scoring_fn=scoring_fn,
            min_vote_amt=min_vote_amt,
            max_vote_amt=max_funding,
            random_seed=seed
        )
    
    results = [run_simulation(ii) for ii in tqdm(range(n_sims), disable=(not verbose))]
    return pd.DataFrame(results)


def run_single_simulation_new(n_badgeholders, badgeholder_expertise_vec, badgeholder_laziness_vec, n_projects, project_impact_vec, quorum, scoring_fn, min_vote_amt, max_vote_amt, max_funding, random_seed):
    np.random.seed(random_seed)
    
    # Create badgeholder population
    badgeholder_list = []
    for ii in range(n_badgeholders):
        badgeholder_id = ii
        agent = QuorumBadgeholder(
            badgeholder_id=badgeholder_id,
            total_funds=max_funding,
            min_vote=min_vote_amt,
            max_vote=max_vote_amt,
            laziness=badgeholder_laziness_vec[ii],
            expertise=badgeholder_expertise_vec[ii],
            coi_factor=0
        )
        badgeholder_list.append(agent)
    badgeholders = QuorumBadgeholderPopulation()
    badgeholders.add_badgeholders(badgeholder_list)
    
    # Create project population
    projects = []
    for ii in range(n_projects):
        project_id = ii
        project = Project(
            project_id=project_id,
            true_impact=project_impact_vec[ii],
            owner_id=None,
        )
        projects.append(project)
    project_population = ProjectPopulation()
    project_population.add_projects(projects)
    
    # Run simulation
    simulation = RoundSimulation()
    simulation.setup_round(badgeholders, project_population)
    simulation.simulate_voting()
    results = simulation.allocate_votes(ThresholdAndAggregate(), quorum=quorum)
    
    return results

